# Shire - Accumulations
## Next year
* Manually

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [4]:
df_export = pd.DataFrame(columns=fix_columns_accumulations[:-1])

# Add HCP
df_export = add_accumulation(df_export, AccType.hcp_amount,
                             registration_fees = '',
                             travel_accommodation = '1,474.96',
                             fees = '6,806.42',
                             related_expenses = '1,254.62',
                             total = '9,536.00'
                            )

# Add HCO
df_export = add_accumulation(df_export, AccType.hco_amount,
                            donations_grants = '',
                            sponsorship = '18,432.00',
                            total = '18,432.00')

# RnD
df_export = add_accumulation(df_export, AccType.rnd,
                            total = '192,168.91')

#Numberize and sum
df_export = remove_comma(df_export)
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

export_acumulations(df_export, 'shire')

hcp_count not found or not unique
hco_count not found or not unique
saved
